# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121625e+02     1.473761e+00
 * time: 0.7589900493621826
     1     1.063489e+01     8.061285e-01
 * time: 1.9988961219787598
     2    -1.203655e+01     9.088353e-01
 * time: 2.7050840854644775
     3    -3.426815e+01     6.992507e-01
 * time: 3.671795129776001
     4    -4.789140e+01     5.503636e-01
 * time: 4.6528401374816895
     5    -5.705792e+01     1.770271e-01
 * time: 5.615707159042358
     6    -5.989271e+01     1.159568e-01
 * time: 6.250575065612793
     7    -6.095307e+01     5.245060e-02
 * time: 6.880446195602417
     8    -6.135074e+01     6.725398e-02
 * time: 7.519620180130005
     9    -6.164022e+01     3.391986e-02
 * time: 8.168438196182251
    10    -6.184724e+01     2.363177e-02
 * time: 8.826052188873291
    11    -6.200394e+01     2.209878e-02
 * time: 9.447591066360474
    12    -6.206315e+01     1.640422e-02
 * time: 10.083823204040527
    13    -6.212272e+01     1.105289e-02
 * time: 10.7306489944458
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557669
    AtomicNonlocal      14.8522640
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666460582
